In [32]:

# https://www.pg4e.com/code/swapi.py
# https://www.pg4e.com/code/myutils.py

# If needed:
# https://www.pg4e.com/code/hidden-dist.py
# copy hidden-dist.py to hidden.py
# edit hidden.py and put in your credentials

# python3 swapi.py
# Pulls data from the swapi.py4e.com API and puts it into our swapi table

import psycopg2
import hidden
import time
import myutils
import requests
import json

def summary(cur) :
    total = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi;')
    todo = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status IS NULL;')
    good = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status = 200;')
    error = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status != 200;')
    print(f'Total={total} todo={todo} good={good} error={error}')

# Load the secrets
secrets = hidden.secrets()

conn = psycopg2.connect(host=secrets['host'],
        port=secrets['port'],
        database=secrets['database'],
        user=secrets['user'],
        password=secrets['pass'],
        connect_timeout=3)

cur = conn.cursor()

defaulturl = 'https://swapi.py4e.com/api/films/1/'
print('If you want to restart the spider, run')
print('DROP TABLE IF EXISTS swapi CASCADE;')
print(' ')

sql = '''
CREATE TABLE IF NOT EXISTS swapi
(id serial, url VARCHAR(2048) UNIQUE, status INTEGER, body JSONB,
created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(), updated_at TIMESTAMPTZ);
'''
print(sql)
cur.execute(sql)

# Check to see if we have urls in the table, if not add starting points
# for each of the object trees
sql = 'SELECT COUNT(url) FROM swapi;'
count = myutils.queryValue(cur, sql)
if count < 1:
    objects = ['films', 'species', 'people']
    for obj in objects:
        sql = f"INSERT INTO swapi (url) VALUES ( 'https://swapi.py4e.com/api/{obj}/1/' )";
        print(sql)
        cur.execute(sql, (defaulturl))
    conn.commit()

many = 0
count = 0
chars = 0
fail = 0
summary(cur)
while True:
    if ( many < 1 ) :
        conn.commit()
        sval = input('How many documents:')
        if ( len(sval) < 1 ) : break
        many = int(sval)

    sql = 'SELECT url FROM swapi WHERE status IS NULL LIMIT 1;'
    url = myutils.queryValue(cur, sql)
    if url is None:
        print('There are no unretrieved documents')
        break

    text = "None"
    try:
        print('=== Url is', url)
        response = requests.get(url)
        text = response.text
        print('=== Text is', text)
        status = response.status_code
        sql = 'UPDATE swapi SET status=%s, body=%s, updated_at=NOW() WHERE url = %s;'
        row = cur.execute(sql, (status, text, url))
        count = count + 1
        chars = chars + len(text)
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page",url)
        print("Error",e)
        fail = fail + 1
        if fail > 5 : break
        continue

    todo = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status IS NULL;')
    print(status, len(text), url, todo)

    # TODO: Add try/except Once we figure out what goes wrong
    js = json.loads(text)

    # Look through all of the "linked data" for other urls to retrieve
    links = ['films', 'species', 'vehicles', 'starships', 'characters']
    for link in links:
        stuff = js.get(link, None)
        if not isinstance(stuff, list) : continue
        for item in stuff:
            sql = 'INSERT INTO swapi (url) VALUES ( %s ) ON CONFLICT (url) DO NOTHING;';
            cur.execute(sql, (item, ))

    many = many - 1
    if count % 25 == 0 :
        conn.commit()
        print(count, 'loaded...')
        time.sleep(1)
        continue

print(' ')
print(f'Loaded {count} documents, {chars} characters')

summary(cur)

print('Closing database connection...')
conn.commit()
cur.close()


If you want to restart the spider, run
DROP TABLE IF EXISTS swapi CASCADE;
 

CREATE TABLE IF NOT EXISTS swapi
(id serial, url VARCHAR(2048) UNIQUE, status INTEGER, body JSONB,
created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(), updated_at TIMESTAMPTZ);

Total=3 todo=3 good=0 error=0
How many documents:5
=== Url is https://swapi.py4e.com/api/films/1/
=== Text is {"title":"A New Hope","episode_id":4,"opening_crawl":"It is a period of civil war.\r\nRebel spaceships, striking\r\nfrom a hidden base, have won\r\ntheir first victory against\r\nthe evil Galactic Empire.\r\n\r\nDuring the battle, Rebel\r\nspies managed to steal secret\r\nplans to the Empire's\r\nultimate weapon, the DEATH\r\nSTAR, an armored space\r\nstation with enough power\r\nto destroy an entire planet.\r\n\r\nPursued by the Empire's\r\nsinister agents, Princess\r\nLeia races home aboard her\r\nstarship, custodian of the\r\nstolen plans that can save her\r\npeople and restore\r\nfreedom to the galaxy....","director":"George Luc

200 496 https://swapi.py4e.com/api/species/4/ 39
=== Url is https://swapi.py4e.com/api/species/5/
=== Text is {"name":"Hutt","classification":"gastropod","designation":"sentient","average_height":"300","skin_colors":"green, brown, tan","hair_colors":"n/a","eye_colors":"yellow, red","average_lifespan":"1000","homeworld":"https://swapi.py4e.com/api/planets/24/","language":"Huttese","people":["https://swapi.py4e.com/api/people/16/"],"films":["https://swapi.py4e.com/api/films/1/","https://swapi.py4e.com/api/films/3/"],"created":"2014-12-10T17:12:50.410000Z","edited":"2014-12-20T21:36:42.146000Z","url":"https://swapi.py4e.com/api/species/5/"}
200 535 https://swapi.py4e.com/api/species/5/ 38
=== Url is https://swapi.py4e.com/api/vehicles/4/
=== Text is {"name":"Sand Crawler","model":"Digger Crawler","manufacturer":"Corellia Mining Corporation","cost_in_credits":"150000","length":"36.8 ","max_atmosphering_speed":"30","crew":"46","passengers":"30","cargo_capacity":"50000","consumables":"2 mont

200 714 https://swapi.py4e.com/api/starships/12/ 27
=== Url is https://swapi.py4e.com/api/starships/13/
=== Text is {"name":"TIE Advanced x1","model":"Twin Ion Engine Advanced x1","manufacturer":"Sienar Fleet Systems","cost_in_credits":"unknown","length":"9.2","max_atmosphering_speed":"1200","crew":"1","passengers":"0","cargo_capacity":"150","consumables":"5 days","hyperdrive_rating":"1.0","MGLT":"105","starship_class":"Starfighter","pilots":["https://swapi.py4e.com/api/people/4/"],"films":["https://swapi.py4e.com/api/films/1/"],"created":"2014-12-12T11:21:32.991000Z","edited":"2014-12-20T21:23:49.889000Z","url":"https://swapi.py4e.com/api/starships/13/"}
200 547 https://swapi.py4e.com/api/starships/13/ 26
=== Url is https://swapi.py4e.com/api/people/2/
=== Text is {"name":"C-3PO","height":"167","mass":"75","hair_color":"n/a","skin_color":"gold","eye_color":"yellow","birth_year":"112BBY","gender":"n/a","homeworld":"https://swapi.py4e.com/api/planets/1/","films":["https://swapi.py4e.com

Unable to retrieve or parse page https://swapi.py4e.com/api/people/13/
Error invalid input syntax for type json
LINE 1: UPDATE swapi SET status=522, body='<!DOCTYPE html>
                                          ^
DETAIL:  Token "<" is invalid.
CONTEXT:  JSON data, line 1: <...



InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [2]:
def summary(cur) :
    total = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi;')
    todo = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status IS NULL;')
    good = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status = 200;')
    error = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status != 200;')
    print(f'Total={total} todo={todo} good={good} error={error}')

In [18]:
# Load the secrets
secrets = hidden.secrets()
conn = psycopg2.connect(host=secrets['host'],
        port=secrets['port'],
        database=secrets['database'],
        user=secrets['user'],
        password=secrets['pass'])
cur = conn.cursor()

In [4]:
defaulturl = 'https://swapi.py4e.com/api/films/1/'
print('If you want to restart the spider, run')
print('DROP TABLE IF EXISTS swapi CASCADE;')
print(' ')

If you want to restart the spider, run
DROP TABLE IF EXISTS swapi CASCADE;
 


In [11]:
sql = '''
SELECT * FROM swapi
'''
print(sql)
cur.execute(sql)


SELECT * FROM swapi



In [12]:
cur.fetchall()

[]

In [20]:
sql = 'SELECT COUNT(url) FROM swapi;'
count = myutils.queryValue(cur, sql) #Nếu chưa có giá trị: count = 0
if count < 1:
    objects = ['films', 'species', 'people']
    for obj in objects:
        sql = f"INSERT INTO swapi (url) VALUES ('https://swapi.py4e.com/api/{obj}/1/')";
        print(sql)
        cur.execute(sql, (defaulturl))
    conn.commit()

INSERT INTO swapi (url) VALUES ('https://swapi.py4e.com/api/films/1/')
INSERT INTO swapi (url) VALUES ('https://swapi.py4e.com/api/species/1/')
INSERT INTO swapi (url) VALUES ('https://swapi.py4e.com/api/people/1/')


In [21]:
many = 0
count = 0
chars =0
fail =0 
summary(cur)
while True:
    if (many <1):
        conn.commit()
        sval = input('How many documents: ')
        if(len(sval) <1): break
        many =int(sval)
    
    sql = 'SELECT url FROM swapi WHERE status IS NULL LIMIT 1;'
    url = myutils.queryValue(cur, sql)
    if url is None:
        print('There are no unretrieved documents')
        break
    text = "None"
    try:
        print('=== Url is', url)
        response = requests.get(url)
        text = response.text
        print('=== Text is', text)
        status= response.status_code
        sql = 'UPDATE swapi SET status=%, body=%, updated_at=NOW() WHERE url=%s;'
        row = cur.execute(sql, (status, text, url))
        count +=1
        chars = chars + len(text)
    except KeyboardInterrupt:
        print('')
        print('Program interrupted by user...')
        break
    except Exception as e:
        print("Unable to retrieve or parse page", url)
        print("Errors", e)
        fail += 1
        if fail > 5: break
        continue
 
    todo = myutils.queryValue(cur, 'SELECT COUNT(*) FROM swapi WHERE status IS NULL;')
    print(status, len(text), url, todo)
    js = json.loads(text)
    links =['films', 'species', 'vehicles', 'starships', 'characters']
    for link in links:
        stuff = js.get(link, None)
        if not isinstance(stuff, list): continue
        for item in stuff:
            sql = 'INSERT INTO swapi (url) VALUES (%s) ON CONFLICT (url) DO NOTHING;'

0

In [23]:
response = requests.get('https://swapi.py4e.com/api/people/1/')
text = response.text
text

'{"name":"Luke Skywalker","height":"172","mass":"77","hair_color":"blond","skin_color":"fair","eye_color":"blue","birth_year":"19BBY","gender":"male","homeworld":"https://swapi.py4e.com/api/planets/1/","films":["https://swapi.py4e.com/api/films/1/","https://swapi.py4e.com/api/films/2/","https://swapi.py4e.com/api/films/3/","https://swapi.py4e.com/api/films/6/","https://swapi.py4e.com/api/films/7/"],"species":["https://swapi.py4e.com/api/species/1/"],"vehicles":["https://swapi.py4e.com/api/vehicles/14/","https://swapi.py4e.com/api/vehicles/30/"],"starships":["https://swapi.py4e.com/api/starships/12/","https://swapi.py4e.com/api/starships/22/"],"created":"2014-12-09T13:50:51.644000Z","edited":"2014-12-20T21:17:56.891000Z","url":"https://swapi.py4e.com/api/people/1/"}'

In [25]:
status = response.status_code
status

200

In [26]:
js = json.loads(text)
js

{'name': 'Luke Skywalker',
 'height': '172',
 'mass': '77',
 'hair_color': 'blond',
 'skin_color': 'fair',
 'eye_color': 'blue',
 'birth_year': '19BBY',
 'gender': 'male',
 'homeworld': 'https://swapi.py4e.com/api/planets/1/',
 'films': ['https://swapi.py4e.com/api/films/1/',
  'https://swapi.py4e.com/api/films/2/',
  'https://swapi.py4e.com/api/films/3/',
  'https://swapi.py4e.com/api/films/6/',
  'https://swapi.py4e.com/api/films/7/'],
 'species': ['https://swapi.py4e.com/api/species/1/'],
 'vehicles': ['https://swapi.py4e.com/api/vehicles/14/',
  'https://swapi.py4e.com/api/vehicles/30/'],
 'starships': ['https://swapi.py4e.com/api/starships/12/',
  'https://swapi.py4e.com/api/starships/22/'],
 'created': '2014-12-09T13:50:51.644000Z',
 'edited': '2014-12-20T21:17:56.891000Z',
 'url': 'https://swapi.py4e.com/api/people/1/'}

In [28]:
stuff = js.get('films', None)
stuff

['https://swapi.py4e.com/api/films/1/',
 'https://swapi.py4e.com/api/films/2/',
 'https://swapi.py4e.com/api/films/3/',
 'https://swapi.py4e.com/api/films/6/',
 'https://swapi.py4e.com/api/films/7/']

True